In [113]:
import pandas as pd 
import numpy as np 
import datetime as dt
data = pd.read_excel('Online Retail.xlsx')
# remove all null data 
data = data[data['CustomerID'].notnull()]
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [114]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [115]:
data['invoiceDAY'] = data['InvoiceDate'].apply(lambda x : dt.datetime(x.year,x.month,x.day))
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoiceDAY
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


In [116]:
pin_date = max(data['invoiceDAY']) + dt.timedelta(1)
pin_date

Timestamp('2011-12-10 00:00:00')

In [117]:
data['Total price'] = data['Quantity'] * data['UnitPrice']
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoiceDAY,Total price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34


In [118]:
new_data = data.groupby('CustomerID').agg({
    'invoiceDAY': lambda x: (pin_date - x.max()).days,
    'InvoiceNo': 'count',
    'Total price': 'sum'
}).reset_index()

new_data.head(5)

,CustomerID,invoiceDAY,InvoiceNo,Total price
0,12346.0,326,2,0.00
1,12347.0,3,182,4310.00
2,12348.0,76,31,1797.24
3,12349.0,19,73,1757.55
4,12350.0,311,17,334.40


In [119]:
def most_purchased_perperson(id):
    list_of_products = dict (data[data['CustomerID'] == id]['Description'].value_counts()[0:1])
    list_of_products = [str(list_of_products.values()).replace('dict_values([','').replace('])',''),str(list_of_products.keys()).replace('dict_keys([','').replace('])','').replace("'","")]
    return list_of_products
print(most_purchased_perperson(12352.0))

['6', 'Manual']


In [120]:
for x in new_data['CustomerID'] :
    new_data.loc[new_data['CustomerID'] == x, 'most_purchased_product'] = most_purchased_perperson(x)[1]

    new_data.loc[new_data['CustomerID'] == x, 'amount_of_product'] = most_purchased_perperson(x)[0]


In [121]:
new_data.head(10)

,CustomerID,invoiceDAY,InvoiceNo,Total price,most_purchased_product,amount_of_product
0,12346.0,326,2,0.00,MEDIUM CERAMIC TOP STORAGE JAR,2
1,12347.0,3,182,4310.00,AIRLINE BAG VINTAGE JET SET BROWN,6
2,12348.0,76,31,1797.24,POSTAGE,4
3,12349.0,19,73,1757.55,PARISIENNE CURIO CABINET,1
4,12350.0,311,17,334.40,CHOCOLATE THIS WAY METAL SIGN,1
5,12352.0,37,95,1545.41,Manual,6
6,12353.0,205,4,89.00,CERAMIC CAKE STAND + HANGING CAKES,1
7,12354.0,233,58,1079.40,JUMBO BAG ALPHABET,1
8,12355.0,215,13,459.40,GROW A FLYTRAP OR SUNFLOWER IN TIN,1
9,12356.0,23,59,2811.43,RED RETROSPOT CAKE STAND,3


In [122]:
new_data['most_purchased_product'].values

array(['MEDIUM CERAMIC TOP STORAGE JAR',
       'AIRLINE BAG VINTAGE JET SET BROWN', 'POSTAGE', ...,
       'FRENCH STYLE STORAGE JAR BONBONS', 'CHARLOTTE BAG SUKI DESIGN',
       'S/4 IVORY MINI ROSE CANDLE IN BOWL'], dtype=object)

In [123]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CustomerID              4372 non-null   float64
 1   invoiceDAY              4372 non-null   int64  
 2   InvoiceNo               4372 non-null   int64  
 3   Total price             4372 non-null   float64
 4   most_purchased_product  4372 non-null   object 
 5   amount_of_product       4372 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 205.1+ KB


In [124]:
new_data.head(5)

,CustomerID,invoiceDAY,InvoiceNo,Total price,most_purchased_product,amount_of_product
0,12346.0,326,2,0.00,MEDIUM CERAMIC TOP STORAGE JAR,2
1,12347.0,3,182,4310.00,AIRLINE BAG VINTAGE JET SET BROWN,6
2,12348.0,76,31,1797.24,POSTAGE,4
3,12349.0,19,73,1757.55,PARISIENNE CURIO CABINET,1
4,12350.0,311,17,334.40,CHOCOLATE THIS WAY METAL SIGN,1


In [125]:
new_data.set_index('CustomerID', inplace=True)


In [126]:
new_data.head(1)

,invoiceDAY,InvoiceNo,Total price,most_purchased_product,amount_of_product
CustomerID,,,,,
12346.0,326,2,0.0,MEDIUM CERAMIC TOP STORAGE JAR,2


## 1 - Standard Scaler Before using KMeans Algorithm

In [127]:
from sklearn.preprocessing import StandardScaler

cols_to_scale = ['invoiceDAY','InvoiceNo','Total price','amount_of_product']

scaler = StandardScaler()
scaled_data = scaler.fit_transform(new_data[cols_to_scale])

scaled_column_names = [col + '_scaled' for col in cols_to_scale]
new_data[scaled_column_names] = scaled_data
x = new_data[scaled_column_names]
new_data.head(5)



,invoiceDAY,InvoiceNo,Total price,most_purchased_product,amount_of_product,invoiceDAY_scaled,InvoiceNo_scaled,Total price_scaled,amount_of_product_scaled
CustomerID,,,,,,,,,
12346.0,326,2,0.00,MEDIUM CERAMIC TOP STORAGE JAR,2,2.316568,-0.391720,-0.231001,-0.331732
12347.0,3,182,4310.00,AIRLINE BAG VINTAGE JET SET BROWN,6,-0.889050,0.382657,0.293432,0.853422
12348.0,76,31,1797.24,POSTAGE,4,-0.164560,-0.266959,-0.012316,0.260845
12349.0,19,73,1757.55,PARISIENNE CURIO CABINET,1,-0.730258,-0.086271,-0.017146,-0.628020
12350.0,311,17,334.40,CHOCOLATE THIS WAY METAL SIGN,1,2.167700,-0.327188,-0.190312,-0.628020


In [128]:
x.head(5)

,invoiceDAY_scaled,InvoiceNo_scaled,Total price_scaled,amount_of_product_scaled
CustomerID,,,,
12346.0,2.316568,-0.391720,-0.231001,-0.331732
12347.0,-0.889050,0.382657,0.293432,0.853422
12348.0,-0.164560,-0.266959,-0.012316,0.260845
12349.0,-0.730258,-0.086271,-0.017146,-0.628020
12350.0,2.167700,-0.327188,-0.190312,-0.628020


## 2 - use KMeans to split the data to 10 groups

In [129]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10,init='k-means++',max_iter=300)

kmeans.fit(x)



C:\Users\ehsan\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10)

In [130]:
def predict_lable(list_of_data):
    d = np.array(list_of_data)

    ss = StandardScaler()
    d = ss.fit_transform(d.reshape(-1, 1))
    d = d.reshape(-1, 4)
    print(d)
    labels_new = kmeans.predict(np.array(d))

    return(labels_new)



In [131]:
new_data['cluster_level']  = kmeans.labels_

In [132]:
new_data.head(10)

,invoiceDAY,InvoiceNo,Total price,most_purchased_product,amount_of_product,invoiceDAY_scaled,InvoiceNo_scaled,Total price_scaled,amount_of_product_scaled,cluster_level
CustomerID,,,,,,,,,,
12346.0,326,2,0.00,MEDIUM CERAMIC TOP STORAGE JAR,2,2.316568,-0.391720,-0.231001,-0.331732,4
12347.0,3,182,4310.00,AIRLINE BAG VINTAGE JET SET BROWN,6,-0.889050,0.382657,0.293432,0.853422,9
12348.0,76,31,1797.24,POSTAGE,4,-0.164560,-0.266959,-0.012316,0.260845,9
12349.0,19,73,1757.55,PARISIENNE CURIO CABINET,1,-0.730258,-0.086271,-0.017146,-0.628020,5
12350.0,311,17,334.40,CHOCOLATE THIS WAY METAL SIGN,1,2.167700,-0.327188,-0.190312,-0.628020,4
12352.0,37,95,1545.41,Manual,6,-0.551616,0.008375,-0.042958,0.853422,9
12353.0,205,4,89.00,CERAMIC CAKE STAND + HANGING CAKES,1,1.115702,-0.383116,-0.220172,-0.628020,0
12354.0,233,58,1079.40,JUMBO BAG ALPHABET,1,1.393588,-0.150803,-0.099662,-0.628020,4
12355.0,215,13,459.40,GROW A FLYTRAP OR SUNFLOWER IN TIN,1,1.214947,-0.344397,-0.175102,-0.628020,0


## 3 - recommendation system

In [158]:
data_of_new_user_for_predict = [326	,2	,0.00,	2]

i = int(predict_lable(data_of_new_user_for_predict)[0])

gr1 = new_data[new_data['cluster_level'] == i]


list_of_products = dict (gr1['most_purchased_product'].value_counts()[0:1])
list_of_products = [str(list_of_products.values()).replace('dict_values([','').replace('])',''),str(list_of_products.keys()).replace('dict_keys([','').replace('])','').replace("'","")]

# list_of_products

gr1 = gr1.reset_index()

print('----------------------------------------------------------------------------')
print('The '+list_of_products[1]+f' hat was offered to customer number[]')
print(f'Group number {str(i+1)}')
print('----------------------------------------------------------------------------')


[[ 1.7320216  -0.57259852 -0.58682457 -0.57259852]]
----------------------------------------------------------------------------
The REGENCY CAKESTAND 3 TIER hat was offered to customer number[]
Group number 5
----------------------------------------------------------------------------


C:\Users\ehsan\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


## خسته نباشید